In [1]:
# 2025.09.18 - train for xBD & mwBTFreddy

%cd /storage/alperengenc/change_detection/ChangeMamba_AG
%ls

import os
# get_ipython().system = os.system

from datetime import datetime
now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# ================= MODEL CONFIG ================= #
cfg_dict = {
    "MambaBDA_Tiny" : '/storage/alperengenc/change_detection/ChangeMamba_AG/changedetection/configs/vssm1/vssm_tiny_224_0229flex.yaml',
    "MambaBDA_Small": '/storage/alperengenc/change_detection/ChangeMamba_AG/changedetection/configs/vssm1/vssm_small_224.yaml',
    "MambaBDA_Base" : '/storage/alperengenc/change_detection/ChangeMamba_AG/changedetection/configs/vssm1/vssm_base_224.yaml',
}
pretrained_dict = {
    "MambaBDA_Tiny" : '/storage/alperengenc/change_detection/ChangeMamba_AG/pretrained_weight/vssm_tiny_0230_ckpt_epoch_262.pth',
    "MambaBDA_Small": '/storage/alperengenc/change_detection/ChangeMamba_AG/pretrained_weight/vssm_small_0229_ckpt_epoch_222.pth',
    "MambaBDA_Base" : '/storage/alperengenc/change_detection/ChangeMamba_AG/pretrained_weight/vssm_base_0229_ckpt_epoch_237.pth',
}

# ================= DATASET CONFIG ================= #
train_dataset_paths_dict = {
    "xBD"           : '/storage/alperengenc/change_detection/xBD_complete_png/train_combined',
    "mwBTFreddy"    : '/storage/alperengenc/change_detection/mwBTFreddy_dataset/mwBTFreddy_v1.0',
    "mwBTFreddy_png": '/storage/alperengenc/change_detection/mwBTFreddy_dataset/mwBTFreddy_v1.0_png',
}
train_data_list_paths_dict = {
    "xBD"           : '/storage/alperengenc/change_detection/xBD_complete_png/train_combined_list.txt',
    "mwBTFreddy"    : '/storage/alperengenc/change_detection/mwBTFreddy_dataset/mwBTFreddy_v1.0/train_list.txt',
    "mwBTFreddy_png": '/storage/alperengenc/change_detection/mwBTFreddy_dataset/mwBTFreddy_v1.0/train_list.txt',
}

test_dataset_paths_dict = {
    "xBD"           : '/storage/alperengenc/change_detection/xBD_complete_png/test',
    "mwBTFreddy"    : '/storage/alperengenc/change_detection/mwBTFreddy_dataset/mwBTFreddy_v1.0',
    "mwBTFreddy_png": '/storage/alperengenc/change_detection/mwBTFreddy_dataset/mwBTFreddy_v1.0_png',
}
test_data_list_paths_dict = {
    "xBD"           : '/storage/alperengenc/change_detection/xBD_complete_png/test_list.txt',
    "mwBTFreddy"    : '/storage/alperengenc/change_detection/mwBTFreddy_dataset/mwBTFreddy_v1.0/test_list.txt',
    "mwBTFreddy_png": '/storage/alperengenc/change_detection/mwBTFreddy_dataset/mwBTFreddy_v1.0/test_list.txt',
}


MODEL_NAME = 'MambaBDA_Tiny' # "MambaBDA_Tiny" || "MambaBDA_Small" || "MambaBDA_Base"
DATASET = "xBD" # "xBD" || "mwBTFreddy" || "mwBTFreddy_png"
EXTENSION = "tif" if DATASET in ["mwBTFreddy"] else "png"

cfg_file = cfg_dict[MODEL_NAME]
pretrained_file = pretrained_dict[MODEL_NAME]
FOCAL = True # True || False

train_dataset_path = train_dataset_paths_dict[DATASET]
train_data_list_path = train_data_list_paths_dict[DATASET]
test_dataset_path = test_dataset_paths_dict[DATASET]
test_data_list_path = test_data_list_paths_dict[DATASET]

logfilename = f"/storage/alperengenc/change_detection/ChangeMamba_AG/LOGLAR_CMAG/train_{now}_{MODEL_NAME}_{DATASET}{'_FOCAL' if FOCAL else ''}.log"
pidfilename = logfilename.replace(".log", ".pid.log")
print(logfilename)
print(pidfilename)


"""
Paper parameters:
- AdamW optimizer //train_MambaBDA.py line 89
- learning rate 1e-4
- weight decay 5e-3
- batch size 16
- # training iterations 50000 //800_000 in github repo example
- training data augmentation: random rotation, left-right & top-bottom flip
"""

COMMAND = f"""
CUDA_LAUNCH_BLOCKING=1 nohup python -u changedetection/script/train_MambaBDA.py \
     --dataset                {DATASET} \
     --batch_size             4 \
     --crop_size              256 \
     --max_iters              200_000 \
     --model_type             {MODEL_NAME} \
     --model_param_path       '/storage/alperengenc/change_detection/ChangeMamba_AG/changedetection/deneme_saved_models/tr{now}_{MODEL_NAME}_{DATASET}{'_FOCAL' if FOCAL else ''}' \
     --train_dataset_path     {train_dataset_path} \
     --train_data_list_path   {train_data_list_path} \
     --test_dataset_path      {test_dataset_path} \
     --test_data_list_path    {test_data_list_path} \
     --cfg                    {cfg_file} \
     --logfile                {logfilename} \
     --pretrained_weight_path {pretrained_file} \
     --learning_rate          0.0001 \
     --weight_decay           0.005 \
     --extension              {EXTENSION} \
     {'--focal_loss' if FOCAL else ''} \
     >/dev/null 2>&1 & echo $! > {pidfilename}
""".replace("\n", "")
#*-- last line alternative for realtime stdout along with file (tee)
# 2>&1 | tee -a {logfilename} & echo $! > {pidfilename}
# >> {logfilename} 2>&1 & echo $! > {pidfilename}

os.system(command=COMMAND)

# BEFORE RERUNNING: check if still running:
# ps -fp $(cat <pidfilename>) || echo "not running"
!ps -fp $(cat {pidfilename}) || echo "not running" && rm {pidfilename}


/storage/alperengenc/change_detection/ChangeMamba_AG
analyze/          kernels/      pipeline_changemamba.ipynb  requirements.txt
changedetection/  LICENSE       pretrained_weight/
classification/   LOGLAR_CMAG/  README.md
figures/          modelcard.sh  README_zh-CN.md
/storage/alperengenc/change_detection/ChangeMamba_AG/LOGLAR_CMAG/train_2025-09-23_09-33-26_MambaBDA_Tiny_xBD_FOCAL.log
/storage/alperengenc/change_detection/ChangeMamba_AG/LOGLAR_CMAG/train_2025-09-23_09-33-26_MambaBDA_Tiny_xBD_FOCAL.pid.log
UID          PID    PPID  C STIME TTY          TIME CMD
alperen+ 2392145       1  0 09:33 ?        00:00:01 python -u changedetection/sc


In [11]:
# 2025.09.20 - inference / test for xBD & mwBTFreddy

%cd /storage/alperengenc/change_detection/ChangeMamba_AG

import os

from datetime import datetime
now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

cfg_dict = {
    "MambaBDA_Tiny" : '/storage/alperengenc/change_detection/ChangeMamba_AG/changedetection/configs/vssm1/vssm_tiny_224_0229flex.yaml',
    "MambaBDA_Small": '/storage/alperengenc/change_detection/ChangeMamba_AG/changedetection/configs/vssm1/vssm_small_224.yaml',
    "MambaBDA_Base" : '/storage/alperengenc/change_detection/ChangeMamba_AG/changedetection/configs/vssm1/vssm_base_224.yaml',
}
test_dataset_paths_dict = {
    "xBD"       : '/storage/alperengenc/change_detection/xBD_complete_png/test',
    "mwBTFreddy": '/storage/alperengenc/change_detection/mwBTFreddy_dataset/mwBTFreddy_v1.0'
}
test_data_list_paths_dict = {
    "xBD"       : '/storage/alperengenc/change_detection/xBD_complete_png/test_list.txt',
    "mwBTFreddy": '/storage/alperengenc/change_detection/mwBTFreddy_dataset/mwBTFreddy_v1.0/all_list.txt'
}

MODEL_NAME = 'MambaBDA_Tiny' # "MambaBDA_Tiny" || "MambaBDA_Small" || "MambaBDA_Base"
DATASET = "xBD" # "xBD" || "mwBTFreddy" || "mwBTFreddy_png"
EXTENSION = "tif" if DATASET in ["mwBTFreddy"] else "png"

cfg_file = cfg_dict[MODEL_NAME]
test_dataset_path = test_dataset_paths_dict[DATASET]
test_data_list_path = test_data_list_paths_dict[DATASET]

# infer_model_path = '/storage/alperengenc/change_detection/ChangeMamba_AG/changedetection/deneme_saved_models/tr2025-09-21_23-51-27_MambaBDA_Tiny_xBD/model_step31250.pth'
infer_model_path = '/storage/alperengenc/change_detection/ChangeMamba_AG/changedetection/deneme_saved_models/tr2025-09-21_17-52-20_MambaBDA_Tiny_xBD/model_step31250.pth'
_, _trained_model_folder, _trained_model_pthname = infer_model_path.rsplit("/", 2)
trained_model_name = f'{_trained_model_folder}'

infer_logfilename = f"/storage/alperengenc/change_detection/ChangeMamba_AG/LOGLAR_CMAG/infer_{trained_model_name}_on_{DATASET}.log"
print(infer_logfilename)
infer_pidfilename = "inferpid.pid"

INFER_COMMAND = f"""
nohup python -u changedetection/script/infer_MambaBDA.py \
    --dataset                   {DATASET}  \
    --type                      'test' \
    --model_type                {MODEL_NAME} \
    --test_dataset_path         {test_dataset_path} \
    --test_data_list_path       {test_data_list_path} \
    --result_saved_path         '/storage/alperengenc/change_detection/ChangeMamba_AG/changedetection/infer_results/in{now}'
    --cfg                       {cfg_file} \
    --resume                    {infer_model_path} \
    --logfile                   {infer_logfilename} \
    --extension                 {EXTENSION} \
    --no-save_output_images \
    >/dev/null 2>&1 & echo $! > {infer_pidfilename}
""".replace("\n", "")

os.system(command=INFER_COMMAND)
!ps -fp $(cat {infer_pidfilename}) || echo "not running" && rm {infer_pidfilename}



/storage/alperengenc/change_detection/ChangeMamba_AG
/storage/alperengenc/change_detection/ChangeMamba_AG/LOGLAR_CMAG/infer_tr2025-09-21_17-52-20_MambaBDA_Tiny_xBD_on_xBD.log
UID          PID    PPID  C STIME TTY          TIME CMD
alperen+ 2045188       1  0 12:02 ?        00:00:01 python -u changedetection/sc
